# Import required libraries

In [91]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
import nltk
from nltk.tokenize import word_tokenize

# Sample answer scripts 

In [92]:
df = pd.read_excel("C://Users//ANESTHESIA//Downloads//questions_answer_key.xlsx")
df.head()

,Qno,Topic,Question in Hindi,Model Answer in Hindi,Question in Marathi,Model Answer in Marathi
0,1,Adaptation,कीड़ों की आंखों को संयुक्त आंखें क्यों कहा जाता है,उनकी आंखों में हजारों लेंस होते हैं,कीटकांच्या डोळ्यांना संयुक्त डोळे का म्हणतात,त्यांच्या डोळ्यात हजारो भिंग असतात
1,2,Adaptation,छलावरण का उपयोग किस लिए किया जाता है,छिपकर शिकार करने के लिए शिकारियों से खुद को ब...,छ्द्मावरणाचा उपयोग प्राणी कशासाठी करतात?,स्वसंरक्षण आणि शिकार
2,3,Adaptation,कुछ फूलों पर रंगीन डिज़ाइन क्यों होता है,ताकि कीड़ों को फूलों में मौजूद रस तक पहुँचने क...,काही फुलांवर रंगीबेरंगी नक्षी का असते,कीटकांना फुलातील रसाचा रस्ता दाखवण्यासाठी
3,4,Adaptation,रेगिस्तान में कम पानी में भी जिंदा रहने के लिए...,पत्ते कांटों में बदल गए हैं,वाळवंटातील झाडांनी कमी पाण्यात जगण्यासाठी पाना...,पानांचे रुपांतर काट्यात झाले
4,5,Circulatory,प्लेटलेट की संख्या कम हो जाने से क्या होगा,जख्म होने पर थक्के ना ज़मने के कारण बहाव ज्याद...,रक्तातील कोणता घटक ऑक्सिजन वाहून नेण्याचे काम ...,लाल रक्तपेशी


# NLP Tasks / Preprocessing

In [93]:
def tokenize_text(text):
    return word_tokenize(text.lower())

In [94]:
def pos_tagging(text):
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    return tagged_tokens

In [95]:
def ner(text):
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    named_entities = ne_chunk(tagged_tokens)
    return named_entities

In [96]:
def grade_response(student_answer, model_answer):
    if student_answer.strip() == model_answer.strip():
        return 1, []  
    else:
        return 0, []  

In [97]:
from nltk import pos_tag
from nltk.chunk import ne_chunk

# Analysis

In [98]:
for index, row in df.iterrows():
    qno = row['Qno']
    student_response_hindi = row['Question in Hindi']
    model_answer_hindi = row['Model Answer in Hindi']
    student_response_marathi = row['Question in Marathi']
    model_answer_marathi = row['Model Answer in Marathi']
    

    pos_tags_hindi = pos_tagging(student_response_hindi)
    named_entities_hindi = ner(student_response_hindi)
    
    pos_tags_marathi = pos_tagging(student_response_marathi)
    named_entities_marathi = ner(student_response_marathi)

    score_hindi, mistakes_hindi = grade_response(student_response_hindi, model_answer_hindi)
    score_marathi, mistakes_marathi = grade_response(student_response_marathi, model_answer_marathi)
    
    # Print the results
    print(f"Question {qno}:")
    print("Hindi - Student Response:", student_response_hindi)
    print("       Model Answer:", model_answer_hindi)
    print("       Hindi Score:", score_hindi)
    print("       Hindi Mistakes:", mistakes_hindi)
    print("       POS Tags (Hindi):", pos_tags_hindi)
    print("       Named Entities (Hindi):", named_entities_hindi)
    print("Marathi - Student Response:", student_response_marathi)
    print("         Model Answer:", model_answer_marathi)
    print("         Marathi Score:", score_marathi)
    print("         Marathi Mistakes:", mistakes_marathi)
    print("         POS Tags (Marathi):", pos_tags_marathi)
    print("         Named Entities (Marathi):", named_entities_marathi)
    print()

Question 1:
Hindi - Student Response: कीड़ों की आंखों को संयुक्त आंखें क्यों कहा जाता है
       Model Answer:  उनकी आंखों में हजारों लेंस होते हैं
       Hindi Score: 0
       Hindi Mistakes: []
       POS Tags (Hindi): [('कीड़ों', 'JJ'), ('की', 'NNP'), ('आंखों', 'NNP'), ('को', 'NNP'), ('संयुक्त', 'NNP'), ('आंखें', 'NNP'), ('क्यों', 'NNP'), ('कहा', 'NNP'), ('जाता', 'NNP'), ('है', 'NN')]
       Named Entities (Hindi): (S
  कीड़ों/JJ
  की/NNP
  आंखों/NNP
  को/NNP
  संयुक्त/NNP
  आंखें/NNP
  क्यों/NNP
  कहा/NNP
  जाता/NNP
  है/NN)
Marathi - Student Response: कीटकांच्या डोळ्यांना संयुक्त डोळे का म्हणतात
         Model Answer:  त्यांच्या डोळ्यात हजारो भिंग असतात
         Marathi Score: 0
         Marathi Mistakes: []
         POS Tags (Marathi): [('कीटकांच्या', 'JJ'), ('डोळ्यांना', 'NNP'), ('संयुक्त', 'NNP'), ('डोळे', 'NNP'), ('का', 'NNP'), ('म्हणतात', 'NN')]
         Named Entities (Marathi): (S
  कीटकांच्या/JJ
  डोळ्यांना/NNP
  संयुक्त/NNP
  डोळे/NNP
  का/NNP
  म्हणतात/NN)

Question 2:
Hindi

#  Grading Student Responses

The set of words that are present in the student response but not in the model answer. In this case, it includes:{} or (0, []) represents a perfect match between the student's response and the model answer, indicating no errors found during evaluation.

In [99]:
for index, row in df.iterrows():
    
    # Extract student response and model answer for each question
    qno = row['Qno']
    student_response_hindi = row['Question in Hindi']
    model_answer_hindi = row['Model Answer in Hindi']
    student_response_marathi = row['Question in Marathi']
    model_answer_marathi = row['Model Answer in Marathi']
    
    score_hindi = grade_response(student_response_hindi, model_answer_hindi)
    score_marathi = grade_response(student_response_marathi, model_answer_marathi)
    
    print(f"Question {qno}:")
    print("Hindi - Student Response:", student_response_hindi)
    print("       Model Answer:", model_answer_hindi)
    print("       Hindi Score:", score_hindi)
    print("Marathi - Student Response:", student_response_marathi)
    print("         Model Answer:", model_answer_marathi)
    print("         Marathi Score:", score_marathi)
    print()

Question 1:
Hindi - Student Response: कीड़ों की आंखों को संयुक्त आंखें क्यों कहा जाता है
       Model Answer:  उनकी आंखों में हजारों लेंस होते हैं
       Hindi Score: (0, [])
Marathi - Student Response: कीटकांच्या डोळ्यांना संयुक्त डोळे का म्हणतात
         Model Answer:  त्यांच्या डोळ्यात हजारो भिंग असतात
         Marathi Score: (0, [])

Question 2:
Hindi - Student Response: छलावरण का उपयोग किस लिए किया जाता है
       Model Answer:  छिपकर शिकार करने के लिए शिकारियों से खुद को बचाने के लिए
       Hindi Score: (0, [])
Marathi - Student Response: छ्द्मावरणाचा उपयोग प्राणी कशासाठी करतात?
         Model Answer: स्वसंरक्षण आणि  शिकार
         Marathi Score: (0, [])

Question 3:
Hindi - Student Response: कुछ फूलों पर रंगीन डिज़ाइन क्यों होता है
       Model Answer:  ताकि कीड़ों को फूलों में मौजूद रस तक पहुँचने का रास्ता दिखे
       Hindi Score: (0, [])
Marathi - Student Response: काही फुलांवर रंगीबेरंगी नक्षी का असते
         Model Answer:  कीटकांना फुलातील रसाचा रस्ता दाखवण्यासाठी
         Mara

In [100]:
for index, row in df.iterrows():

    student_response_hindi = row['Question in Hindi']
    model_answer_hindi = row['Model Answer in Hindi']

    score_hindi = grade_response(student_response_hindi, model_answer_hindi)
    scores.append(score_hindi)

In [101]:
# Creating datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [102]:
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

In [103]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\ANESTHESIA\anaconda3\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Hardware Selection

In [104]:
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [105]:
for epoch in range(3):  
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


C:\Users\ANESTHESIA\AppData\Local\Temp\ipykernel_13712\2210648296.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


# Evaluating the model

In [106]:
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predicted = torch.argmax(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


if total == 0:
    accuracy = "Undefined"
else:
    accuracy = int(correct / total * 100)  # Convert accuracy to an integer percentage
print(f'Accuracy: {accuracy}')

C:\Users\ANESTHESIA\AppData\Local\Temp\ipykernel_13712\875721199.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Accuracy: 100
